<a href="https://colab.research.google.com/github/Khalil1232/Machine-Learning-ULaval/blob/main/Nonlinear_discriminants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Devoir 3, Question 2 : Discriminants non-linéaires

# Homework 3, Question 2: Nonlinear discriminants

## Code préambule

## Preamble code

In [ ]:
import numpy
import time
import pandas
pandas.set_option('display.max_colwidth', 0)
import collections
import itertools

from IPython import display

from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import minmax_scale
from sklearn.svm import SVC

# Nous ne voulons pas être signalé par ce type d'avertissement, non pertinent pour le devoir
# We don't want to be signaled of this warning, irrelevant for the homework
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Fonction pour vérifier le temps d'exécution
# Function to verify execution time
_times = []
def checkTime(maxduration, question):
    duration = _times[-1] - _times[-2]
    if duration > maxduration:
        print("[ATTENTION] Votre code pour la question {0} met trop de temps à s'exécuter! ".format(question)+
              "Le temps maximum permis est de {0:.4f} secondes, ".format(maxduration)+
              "mais votre code a requis {0:.4f} secondes! ".format(duration)+
              "Assurez-vous que vous ne faites pas d'appels bloquants (par exemple à show()) dans cette boucle!")

# Définition des durées d'exécution maximales pour chaque classifieur
# Definition of maximum execution time for each classifier
TMAX_KNN = 40
TMAX_SVM = 200
TMAX_MLP = 400
TMAX_EVAL = 80

## Q2A
Soit les classifieurs suivants, présentés en classe :
- Classement par les $k$-plus proches voisins avec distance euclidienne (`neighbors.KNeighborsClassifier`);
- Séparateurs à vastes marges avec noyau gaussien (`svm.SVC`), notez que le paramètre du noyau de la fonction est $\gamma=\frac{1}{2\sigma^2}$;
- Perceptron multicouche pour le classement (`neural_network.MLPClassifier`). Pour cette méthode, utilisez le paramètre `max_iter=100`.

Pour les trois algorithmes présentés ci-haut, déterminez les hyperparamètres importants qui peuvent influencer significativement les résultats d’apprentissage (limitez-vous à un maximum de deux hyperparamètres importants par algorithme). Dans le tableau de réponse, affichez les deux hyperparamètres choisis. Donnez également la plage de valeurs qui devrait être testée pour chacun de ces hyperparamètres afin de faire un bon ajustement de la configuration des classifieurs sur le jeu Pendigits. Vous testerez ces plages dans un prochain exercice.

Le jeu de données [Pendigits](https://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits) vise à reconnaître des chiffres manuscrits (10 classes) à partir des coordonnées de 8 points des tracés des caractères (16 variables). Le jeu de données est [disponible directement sur PAX](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/ucipendigits.npy).

## Q2A
Consider the following classifiers, presented in class:
- Classification by $k$-nearest neighbors with Euclidean distance (`neighbors.KNeighborsClassifier`);
- Support vector machine with Gaussian kernel (`svm.SVC`), note that the kernel parameter of the function is $\gamma=\frac{1}{2\sigma^2}$;
- Multilayer perceptron for classification (`neural_network.MLPClassifier`). For this method, use the parameter `max_iter = 100`.

For the three algorithms presented above, determine the important hyperparameters that can significantly influence learning outcomes (limit yourself to a maximum of two important hyperparameters per algorithm). In the answer table, display the 2 chosen hyperparameters. Also give the range of values that should be tested for each of these hyperparameters in order to make a good fit for the configuration of the classifiers on the Pendigits game. You will test these ranges in a future exercise.

The [Pendigits](https://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits) dataset aims at recognizing handwritten digits (10 classes) from the coordinates of 8 points taken from the character plots (16 variables). The dataset is [available directly on PAX](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/ucipendigits.npy).

### Entrez votre solution à Q2A dans la cellule ci-dessous
Rapportez les paramètres d’entraînement et les plages de valeurs correspondantes que vous avez choisis pour chacun des algorithmes, en modifiant le tableau donné ci-dessous, dans la cellule de solution.
~~~md
Algorithme | Parametre 1    | Parametre 2    | Plage param. 1 |Plage param. 2 |
-----------|----------------|----------------|----------------|---------------|
KNN        | -              | -              | -              | -             |
SVM        | -              | -              | -              | -             |
MLP        | -              | -              | -              | -             |
~~~

### Enter your answer to Q2A in the cell below
Report the training parameters and the corresponding ranges of values you have chosen for each of the algorithms, by modifying the table structure below into the solution cell.
~~~md
Algorithm  | Parameter 1    | Parameter 2    | Range param. 1 |Range param. 2 |
-----------|----------------|----------------|----------------|---------------|
KNN        | -              | -              | -              | -             |
SVM        | -              | -              | -              | -             |
MLP        | -              | -              | -              | -             |
~~~

<div class="feedback-cell" style="background: rgba(100 , 100 , 100 , 0.4)">
                <h3>Votre soumission a été enregistrée!</h3>
                <ul>
                    <li>notez qu'il n'y a <strong>pas</strong> de correction automatique pour cet exercice&puncsp;;</li>
                    <li>par conséquent, votre note est <strong>actuellement</strong> zéro&puncsp;;</li>
                    <li>elle sera cependant ajustée par le professeur dès que la correction manuelle sera complétée&puncsp;;</li>
                    <li>vous pouvez soumettre autant de fois que nécessaire jusqu'à la date d'échéance&puncsp;;</li>
                    <li>mais évitez de soumettre inutilement.</li>
                </ul>
            </div>

Algorithme | Parametre 1            | Parametre 2         | Plage param. 1                 |Plage param. 2 |
-----------|------------------------|---------------------|--------------------------------|
  KNN      | -number of neighbors   | -weights            | - k=1----->55                |-['uniform','distance']     
SVM        | -kernel                | -the penalty C      | - [‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’]  |[100, 10, 1.0, 0.1, 0.001]
MLP        | -hidden layer sizes    | -activation function| - n=3---->100             |['identity','logistic','relu','tanh']

## Q2B
Tester les discriminants de la question Q2A avec le jeu de données Pendigits. Normalisez les données dans $[0,1]^D$ avant vos traitements (`preprocessing.minmax_scale`). Utilisez 5500 instances choisies au hasard comme données d’entraînement et les 5492 données restantes pour les tests. Notez que la dernière colonne du jeu de données correspond aux étiquettes de classe.

## Q2B
Test the discriminants of question Q2A with the Pendigits dataset. Normalize the data in $[0,1]^D$ before your processing (`preprocessing.minmax_scale`). Use 5500 randomly selected instances for training data and the remaining 5492 data for testing. Note that the last column of the dataset corresponds to the class labels.

### Entrez votre solution à Q2B dans la cellule ci-dessous

### Enter your answer to Q2B in the cell below

In [ ]:
# Initialisation du jeu de données Pendigits
# Initializing Pendigits dataset
print('Lecture du jeu de données Pendigits...')
filePendigits = '/pax/shared/GIF-4101-7005/ucipendigits.npy'
dataPendigits = numpy.load(filePendigits)
data, target = dataPendigits[:, :-1].astype('float32'), dataPendigits[:, -1]
print('Lecture terminé.')
print('En cours...')
# *** TODO ***
# Normalisez les données d'entrée entre 0 et 1 pour toutes les dimensions.
# Notez que la fonction retourne les données d'une manière différente des
# fonctions load_* que vous avez déjà utilisées (ex. load_iris(),
# load_breast_cancer()). Assurez-vous que vous utilisez correctement les
# données et qu'elles sont du bon type!
# Normalize the input data between 0 and 1 for all dimensions.
# Note that the function returns the data in a different way than the load_*
# functions you you have used before (e.g. load_iris(), load_breast_cancer()).
# Make sure you are using the data correctly and that it is of the right type!
y = target
X = minmax_scale(data)

# Séparez le jeu de données Pendigits en deux sous-jeux: entraînement (5500)
# et test (reste des données). Pour la suite du code, rappelez-vous que vous ne
# pouvez PAS vous servir du jeu de test pour déterminer la configuration
# d'hyper-paramètres la plus performante. Ce jeu de test ne doit être utilisé qu'à
# la toute fin, pour rapporter les résultats finaux en généralisation.
# Split the Pendigits dataset into two subsets: training (5500) and testing
# (the rest of the data). For the remainder of the code, remember that you
# cannot use the test set to determine the most appropriate hyper-parameter
# configuration. This test set should only be used at the very end, to report the
# final results in generalization.
# ******
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5500, test_size=len(y)-5500, random_state=0)
# Création du tableau pour accumuler les résultats
# Creation of the table to accumulate results
results = {'Classifiers':[],
           'Optimal_hp1':[],
           'Optimal_hp2':[],
           'Time_train':[],
           'Time_test':[],
           'Score_train':[],
           'Score_test':[],
          }

_times.append(time.time())

# Pour chaque classifieurs (k-PPV, SVM à noyau gaussien, Perceptron multicouche),
# déterminez les valeurs optimales des hyperparamètres à utiliser. Suivez les
# instructions de l'énoncé de Q2A et Q2B quant au nombre d'hyperparamètres à optimiser.
# Vous êtes libres d'utiliser la méthodologie que vous souhaitez, en autant que
# vous ne touchez pas au jeu de test pour l'optimisation.
# For each classifier (k-NN, Gaussian kernel SVM, Multi-layer Perceptron),
# determine the optimal values of the hyperparameters to use. Follow the
# instructions in the statement of Q2A and Q2B regarding the number of
# hyperparameters to optimize. You are free to use any methodology you wish, as
# long as you do not make use of the test set for optimization.

# -------
# Optimisez la paramétrisation du k-PPV
# Optimize k-NN parametrization

# *** TODO ***
# Ajustez les hyperparamètres du k-PPV
# Adjust k-NN hyperparamters
k_val = numpy.arange(1,55)
weights = ['uniform', 'distance']
scores = numpy.zeros((len(k_val), len(weights)))
score_max = 0
opt_k=0
opt_w=0
for i in k_val:
    for j in weights:
        score = 0
        XX_train, XX_test, yy_train, yy_test = train_test_split(X_train, y_train, train_size=0.7, test_size=0.3, random_state=0)
        clf_knn = KNeighborsClassifier(n_neighbors= i , weights= j)
        clf_knn.fit(XX_train, yy_train)
        score = clf_knn.score(XX_test, yy_test)
        if score > score_max:
            score_max = score
            opt_k = i
            opt_w = j

# ******

clf_name = clf_knn.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***
# Calculez le temps d'entraînement pour le k-PPV
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the k-NN
# and put it in the time_train variable by
# replacing the 0.
clf_knn = KNeighborsClassifier(n_neighbors= opt_k , weights= opt_w)
t_deb = time.time()
clf_knn.fit(X_train, y_train)
duration = numpy.round_(time.time() - t_deb, 3)
score = clf_knn.score(X_train, y_train)
time_train = duration
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le k-PPV
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the k-NN
# and put it in the score_train variable by
# replacing the 0.
score_train = score
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = opt_k
optimal_hp2 = opt_w
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_KNN, "k-plus proches voisins")
print('fin KNeighborsClassifier')

# -------

# Optimisez la paramétrisation du SVM à noyau gaussien
# Optimize Gaussian kernel SVM parametrization

# *** TODO ***
# Ajustez les hyperparamètres du SVM
# Adjust SVM hyperparamters
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
C = [100, 10, 1.0, 0.1, 0.001]
scores = numpy.zeros((len(kernels), len(C)))
score_max = 0
opt_ker='linear'
opt_C=0
for i in kernels:
    for j in C:
        score = 0
        XX_train, XX_test, yy_train, yy_test = train_test_split(X_train, y_train, train_size=0.7, test_size=0.3, random_state=0)
        clf_svc = SVC(kernel = i, C = j)
        clf_svc.fit(XX_train, yy_train)
        score = clf_svc.score(XX_test, yy_test)
        if score > score_max:
            score_max = score
            opt_ker = i
            opt_C = j


# ******

clf_name = clf_svc.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***
# Calculez le temps d'entraînement pour le SVM
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the SVM
# and put it in the time_train variable by
# replacing the 0.
clf_svc = SVC(kernel = opt_ker, C = opt_C)
t_deb = time.time()
clf_svc.fit(X_train, y_train)
duration = numpy.round_(time.time() - t_deb, 3)
score = clf_svc.score(X_train, y_train)
time_train = duration

# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le SVM
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the SVM
# and put it in the score_train variable by
# replacing the 0.
score_train = score
# ******

results['Score_train'].append(score_train)
# *** TODO ***
# Stockez vos valeurs optimales
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = opt_ker
optimal_hp2 = opt_C
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_SVM, "Support vector machine")
print('fin SVM')

# -------

# Optimisez la paramétrisation du perceptron multicouche
# Optimize multilayer perceptron parametrization

# *** TODO ***
# Ajustez les hyperparamètres du MLP
# Adjust MLP hyperparamters
neurons = numpy.arange(3,20)
act_func = ['identity','logistic','relu','tanh']
scores = numpy.zeros((len(neurons), len(act_func)))
score_max = 0
opt_size= 3
opt_func= 'identity'
for i in neurons:
    for j in act_func:
        score = 0
        XX_train, XX_test, yy_train, yy_test = train_test_split(X_train, y_train, train_size=0.7, test_size=0.3, random_state=0)
        clf_mlp = MLPClassifier(hidden_layer_sizes = opt_size, activation = opt_func)
        clf_mlp.fit(XX_train, yy_train)
        score = clf_mlp.score(XX_test, yy_test)
        if score > score_max:
            score_max = score
            opt_size = i
            opt_func = j


# ******

clf_name = clf_mlp.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)
# *** TODO ***
# Calculez le temps d'entraînement pour le PMC
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the MLP
# and put it in the time_train variable by
# replacing the 0.
clf_mlp = MLPClassifier(hidden_layer_sizes = opt_size, activation = opt_func)
t_deb = time.time()
clf_mlp.fit(X_train, y_train)
duration = numpy.round_(time.time() - t_deb, 3)
score = clf_mlp.score(X_train, y_train)
time_train = duration

# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le PMC
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the MLP
# and put it in the score_train variable by
# replacing the 0.
score_train = score
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = opt_size
optimal_hp2 = opt_func
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_MLP, "Perceptron multicouche")
print('fin Perceptron multicouche')

# -------

# Après avoir bien recherché les meilleures paramétrisations
# pour vos discriminants, évaluez leurs performances sur le
# jeu de test. Rapportez les performances dans le tableau généré.

# k-PPV / k-NN

# *** TODO ***
# Calculez le temps d'inference pour le k-PPV
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the k-NN
# and put it in the time_test variable by
# replacing the 0.

t_deb = time.time()
score = clf_knn.score(X_test, y_test)
duration = numpy.round_(time.time() - t_deb, 3)

time_test = duration


# Calculez le score en test pour le k-PPV
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the k-NN
# and put it in the score_test variable by
# replacing the 0.
score_test = score
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

# SVM

# *** TODO ***
# Calculez le temps d'inference pour le SVM
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the SVM
# and put it in the time_test variable by
# replacing the 0.
t_deb = time.time()
score = clf_svc.score(X_test, y_test)
duration = numpy.round_(time.time() - t_deb, 3)

time_test = duration

# Calculez le score en test pour le SVM
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the SVM
# and put it in the score_test variable by
# replacing the 0.
score_test = score
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)
# PMC / MLP

# *** TODO ***
# Calculez le temps d'inference pour le PMC
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the MLP
# and put it in the time_test variable by
# replacing the 0.
t_deb = time.time()
score = clf_mlp.score(X_test, y_test)
duration = numpy.round_(time.time() - t_deb, 3)

time_test = duration

# Calculez le score en test pour le PMC
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the MLP
# and put it in the score_test variable by
# replacing the 0.
score_test = score
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

_times.append(time.time())
checkTime(TMAX_EVAL, "Evaluation des modèles")

# Affichage des résultats
# Display results
df = pandas.DataFrame(results)
display.display(df)


Lecture du jeu de données Pendigits...
Lecture terminé.
En cours...
fin KNeighborsClassifier
fin SVM
fin Perceptron multicouche


,Classifiers,Optimal_hp1,Optimal_hp2,Time_train,Time_test,Score_train,Score_test
0,KNeighborsClassifier,2,uniform,0.001,0.524,0.995455,0.989803
1,SVC,rbf,10,0.111,0.187,0.998000,0.995448
2,MLPClassifier,8,logistic,2.117,0.002,0.896000,0.887109


### Patron de code réponse à l'exercice Q2B

### Q2B answer code template

In [ ]:
# Initialisation du jeu de données Pendigits
# Initializing Pendigits dataset
print('Lecture du jeu de données Pendigits...')
filePendigits = '/pax/shared/GIF-4101-7005/ucipendigits.npy'
dataPendigits = numpy.load(filePendigits)
data, target = dataPendigits[:, :-1].astype('float32'), dataPendigits[:, -1]
print('Lecture terminé.')

# *** TODO ***
# Normalisez les données d'entrée entre 0 et 1 pour toutes les dimensions.
# Notez que les données sont retournées d'une manière différente des
# fonctions load_* que vous avez déjà utilisées (ex. load_iris(),
# load_breast_cancer()). Assurez-vous que vous utilisez correctement les
# données et qu'elles sont du bon type!
# Normalize the input data between 0 and 1 for all dimensions.
# Note that that the data are provided in a different way than the load_*
# functions you you have used before (e.g. load_iris(), load_breast_cancer()).
# Make sure you are using the data correctly and that it is of the right type!

# Séparez le jeu de données Pendigits en deux sous-jeux: entraînement (5500)
# et test (reste des données). Pour la suite du code, rappelez-vous que vous ne
# pouvez PAS vous servir du jeu de test pour déterminer la configuration
# d'hyper-paramètres la plus performante. Ce jeu de test ne doit être utilisé qu'à
# la toute fin, pour rapporter les résultats finaux en généralisation.
# Split the Pendigits dataset into two subsets: training (5500) and testing
# (the rest of the data). For the remainder of the code, remember that you
# cannot use the test set to determine the most appropriate hyper-parameter
# configuration. This test set should only be used at the very end, to report the
# final results in generalization.
# ******

# Création du tableau pour accumuler les résultats
# Creation of the table to accumulate results
results = {'Classifiers':[],
           'Optimal_hp1':[],
           'Optimal_hp2':[],
           'Time_train':[],
           'Time_test':[],
           'Score_train':[],
           'Score_test':[],
          }

_times.append(time.time())

# Pour chaque classifieurs (k-PPV, SVM à noyau gaussien, Perceptron multicouche),
# déterminez les valeurs optimales des hyperparamètres à utiliser. Suivez les
# instructions de l'énoncé de Q2A et Q2B quant au nombre d'hyperparamètres à optimiser.
# Vous êtes libres d'utiliser la méthodologie que vous souhaitez, en autant que
# vous ne touchez pas au jeu de test pour l'optimisation.
# For each classifier (k-NN, Gaussian kernel SVM, Multi-layer Perceptron),
# determine the optimal values of the hyperparameters to use. Follow the
# instructions in the statement of Q2A and Q2B regarding the number of
# hyperparameters to optimize. You are free to use any methodology you wish, as
# long as you do not make use of the test set for optimization.

# -------

# Optimisez la paramétrisation du k-PPV
# Optimize k-NN parametrization

# *** TODO ***
# Ajustez les hyperparamètres du k-PPV
# Adjust k-NN hyperparamters
clf_knn = KNeighborsClassifier()
# ******

clf_name = clf_knn.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***
# Calculez le temps d'entraînement pour le k-PPV
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the k-NN
# and put it in the time_train variable by
# replacing the 0.
time_train = 0
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le k-PPV
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the k-NN
# and put it in the score_train variable by
# replacing the 0.
score_train = 0
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = 0
optimal_hp2 = 0
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_KNN, "k-plus proches voisins")

# -------

# Optimisez la paramétrisation du SVM à noyau gaussien
# Optimize Gaussian kernel SVM parametrization

# *** TODO ***
# Ajustez les hyperparamètres du SVM
# Adjust SVM hyperparamters
clf_svc = SVC()
# ******

clf_name = clf_svc.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***
# Calculez le temps d'entraînement pour le SVM
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the SVM
# and put it in the time_train variable by
# replacing the 0.
time_train = 0
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le SVM
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the SVM
# and put it in the score_train variable by
# replacing the 0.
score_train = 0
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = 0
optimal_hp2 = 0
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_SVM, "Support vector machine")

# -------

# Optimisez la paramétrisation du perceptron multicouche
# Optimize multilayer perceptron parametrization

# *** TODO ***
# Ajustez les hyperparamètres du MLP
# Adjust MLP hyperparamters
clf_mlp = MLPClassifier()
# ******

clf_name = clf_mlp.__class__.__name__
if clf_name not in results['Classifiers']:
    results['Classifiers'].append(clf_name)

# *** TODO ***
# Calculez le temps d'entraînement pour le PMC
# et mettez-le dans la variable time_train en
# remplaçant le 0.
# Calculate the training time for the MLP
# and put it in the time_train variable by
# replacing the 0.
time_train = 0
# ******

results['Time_train'].append(time_train)

# *** TODO ***
# Calculez le score d'entraînement pour le PMC
# et mettez-le dans la variable score_train en
# remplaçant le 0.
# Calculate the training score for the MLP
# and put it in the score_train variable by
# replacing the 0.
score_train = 0
# ******

results['Score_train'].append(score_train)

# *** TODO ***
# Stockez vos valeurs optimales
# d'hyperparamètres dans les variables
# optimal_hp1 et optimal_hp2 en remplaçant
# le 0.
# Store your optimal hyperparameters values
# in the variables optimal_hp1 and optimal_hp2 by
# replacing the 0.
optimal_hp1 = 0
optimal_hp2 = 0
# ******

results['Optimal_hp1'].append(optimal_hp1)
results['Optimal_hp2'].append(optimal_hp2)
_times.append(time.time())
checkTime(TMAX_MLP, "Perceptron multicouche")

# -------

# Après avoir bien recherché les meilleures paramétrisations
# pour vos discriminants, évaluez leurs performances sur le
# jeu de test. Rapportez les performances dans le tableau généré.

# k-PPV / k-NN

# *** TODO ***
# Calculez le temps d'inference pour le k-PPV
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the k-NN
# and put it in the time_test variable by
# replacing the 0.
time_test = 0

# Calculez le score en test pour le k-PPV
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the k-NN
# and put it in the score_test variable by
# replacing the 0.
score_test = 0
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

# SVM

# *** TODO ***
# Calculez le temps d'inference pour le SVM
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the SVM
# and put it in the time_test variable by
# replacing the 0.
time_test = 0

# Calculez le score en test pour le SVM
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the SVM
# and put it in the score_test variable by
# replacing the 0.
score_test = 0
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

# PMC / MLP

# *** TODO ***
# Calculez le temps d'inference pour le PMC
# et mettez-le dans la variable time_test en
# remplaçant le 0.
# Calculate the inference time for the MLP
# and put it in the time_test variable by
# replacing the 0.
time_test = 0

# Calculez le score en test pour le PMC
# et mettez-le dans la variable score_test en
# remplaçant le 0.
# Calculate the test score for the MLP
# and put it in the score_test variable by
# replacing the 0.
score_test = 0
# ******

results['Time_test'].append(time_test)
results['Score_test'].append(score_test)

_times.append(time.time())
checkTime(TMAX_EVAL, "Evaluation des modèles")

# Affichage des résultats
# Display results
df = pandas.DataFrame(results)
display.display(df)